In [5]:
import dask.array as da
import numpy as np
from dask_glm.algorithms import newton, admm, gradient_descent # the only three we should advertise right now
from dask_glm.utils import sigmoid

## for comparisons
from statsmodels.discrete.discrete_model import Logit
from statsmodels.genmod.generalized_linear_model import GLM

from statsmodels.base.data import  handle_data

X = np.random.normal(size=(1000, 25))
beta = np.random.normal(size=25)
beta *= np.random.randint(0, 2, size=25)
y = X.dot(beta) + np.random.normal(size=1000)
m = 5

def make_data(N, p, scale=1.0, corr=0.25):
    N = int(N)
    beta = (np.random.random(p-1) - 0.5) * 2
    beta = np.append(beta, scale)
    
    X = np.ones((N, p))
    X[:,:2] = np.random.multivariate_normal([0,0], [[1, corr], [corr, 1]], size=N)
    X[:, 2:-1] = (np.random.random((N, p-3)) - 0.5) * 2
    
    Xbeta = X.dot(beta)
    
    y = np.random.random(Xbeta.shape) < sigmoid(Xbeta)
    return X, y, beta

X, y, true_beta = make_data(1e4, 10, scale=1.25, corr=0.25)
X = da.from_array(X, chunks=(1e3, 10))
y = da.from_array(y, chunks=(1e3,))

# Use statsmodels on subset
X_, y_ = X[:1000].compute(), y[:1000].compute()

mod = GLM(y_, X_)
res = mod.fit(method='nm')
res.params

/Users/tom.augspurger/Envs/py3/lib/python3.6/site-packages/statsmodels/statsmodels/base/model.py:509: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


array([ 0.11351132, -0.0064103 ,  0.05055113,  0.17582365,  0.16448483,
        0.12441961, -0.02800043, -0.10632769, -0.08254563,  0.71340299])

In [8]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                 1000
Model:                            GLM   Df Residuals:                      990
Model Family:                Gaussian   Df Model:                            9
Link Function:               identity   Scale:                  0.162687326622
Method:                            nm   Log-Likelihood:                -505.95
Date:                Mon, 13 Mar 2017   Deviance:                       161.06
Time:                        07:24:52   Pearson chi2:                     161.
No. Iterations:                   100                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1135      0.013      8.700      0.000       0.088       0.139
x2            -0.0064      0.013     -0.488      0.626      -0.032       0.019
x3             0.0506      0.022      2.331      0.020       0.008       0.093
x4             0.1758      0.023      7.705      0.000       0.131       0.221
x5             0.1645      0.022      7.496      0.000       0.121       0.207
x6             0.1244      0.022      5.581      0.000       0.081       0.168
x7            -0.0280      0.023     -1.239      0.215      -0.072       0.016
x8            -0.1063      0.022     -4.922      0.000      -0.149      -0.064
x9            -0.0825      0.022     -3.800      0.000      -0.125      -0.040
const          0.7134      0.013     55.761      0.000       0.688       0.738
==============================================================================
"""

In [19]:
from statsmodels.genmod.generalized_linear_model import DaskGLM

In [23]:
dglm = DaskGLM(y, X)
dres = dglm.fit()
dres.params

Converged! 29


array([ 0.77925663,  0.0133706 ,  0.39271345,  0.92552116,  0.77705228,
        0.96426037, -0.48653848, -0.55960016, -0.18188098,  1.23497578])